# Imports

In [2]:
%pip install --upgrade tiktoken
%pip install --upgrade sentencepiece
%pip install --upgrade tensorflow
%pip install --upgrade tf_keras
%pip install --upgrade transformers
%pip install ml_dtypes==0.5.1 # it appears some code does not work if I don't downgrade this library
%pip install --upgrade minicons

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#import tensorflow
#import pandas as pd
import numpy as np
#from transformers import pipeline
#from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
#import torch
import huggingface_hub as hgh
from minicons import scorer
import operator

c:\Users\borah\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Login (for HuggingFace)

In [4]:
login_token_file = open("access_items/hg_token.txt", "r")
token = login_token_file.readline()
login_token_file.close()
hgh.login(token)

# Loading data for testing

In [5]:
def load_sentences(filename):
    """Loads sentences from a file formatted as: 
    context,translation
        completion1,translation,is_grammatical
        completion2,translation,is_grammatical
        ...
    """
    out = dict()
    with open(filename, "r", encoding="utf-8") as file:
        cur_header = None
        blank_index = int(1)
        for raw_line in file:
            if (raw_line.lstrip().startswith("#")):
                continue
            if (len(raw_line.strip()) == 0):
                continue
            line = raw_line.rstrip()
            if (line == "\\nil"):
                cur_header = blank_index
                out[blank_index] = []
                blank_index += 1
            elif line.startswith("\t") or line.startswith(" "*4):
                if not cur_header:
                    raise Exception(f"File '{filename}' in incorrect format at line {line}.")
                out[cur_header].append(tuple(line.lstrip().split("::")))
            else:
                translation_pair = tuple(line.split("::"))
                out[translation_pair] = []
                cur_header = translation_pair
    return out

In [6]:
## the sentence sets for English (just to test as control)
english_contextual_sentences = {
    ("A boy and a girl walked into the classroom.",)*2 : [("A boy sat down.",)*2, ("The boy sat down.",)*2],
    ("I am expecting a student to come to class today.",)*2 : [("Has a student arrived?",)*2, ("Has the student arrived?",)*2],
    ("There once was a massive tree in the forest.",)*2 : [("A tree fell down.",)*2, ("The tree fell down.",)*2],
    ("Two men were talking.",)*2 : [("The man said a joke.",)*2, ("One man said a joke.",)*2]
}

## load in assamese sentence sets
assamese_contextual_sentences = load_sentences("test_sentences/assamese.txt")
assamese_contextual_cls_sentences = load_sentences("test_sentences/assamese_classifiers.txt")
print(assamese_contextual_sentences)
print(assamese_contextual_cls_sentences)

{('এজন ল’ৰা আৰু এজনী ছোৱালী এটা কোঠাটোলৈ সোমাই গ’ল।', 'A boy and a girl entered a room.'): [('ল’ৰাজন বহিলে।', 'The boy sat down.', '1'), ('এজন ল’ৰা বহিলে।', 'A boy sat down.', '0')], ('টেবুলখনত দুটা মেকুৰী বহি আছে।', 'There are two cats sitting on the table.'): [('এটা মেকুৰী তললৈ জপিয়াই পৰে।', 'One cat jumps down.', '1'), ('মেকুৰীটোৱে তললৈ জপিয়াই পৰে।', 'The cat jumps down.', '0')], ('দুজন মানুহে কথা পাতি আছিলে।', 'Two men were talking.'): [('এজন মানুহে গান গাবলৈ ধৰিলে।', 'One man started to sing.', '1'), ('মানুহজনে গান গাবলৈ ধৰিলে।', 'The man started to sing.', '0')], ('দুটা কেক আৰু দুটা কুকিজ।', 'I bought two cakes and two cookies.'): [('কুকিজ দুটা পিছত খাম।', "I'll eat the two cookies later.", '1'), ('দুটা কুকিজ পিছত খাম।', "I'll eat two cookies later.", '0')], ('কুকুৰটোৰ চাৰিটা পোৱালি আছিলে', 'The dog has four puppies'): [('কিন্তু দুটা পোৱালি মৰিলে।', 'but two puppies died.', '1'), ('কিন্তু পোৱালি দুটা মৰিলে।', 'but the two puppies died.', '0')], 1: [('সূৰ্য্যতুই পোহৰ দিয়ে।', 'T

In [7]:
classifier_sent_count = sum([len(val) for (_, val) in assamese_contextual_cls_sentences.items()])
definiteness_sent_count = sum([len(val) for (_, val) in assamese_contextual_sentences.items()])
print(f"Length(CLS):{classifier_sent_count}, Length(DEF):{definiteness_sent_count}")

Length(CLS):34, Length(DEF):28


(Citations)

* (<em>AxomiyaBERTa</em>) [AxomiyaBERTa: A Phonologically-aware Transformer Model for Assamese](https://aclanthology.org/2023.findings-acl.739/) (Nath et al., Findings 2023) 

* (<em>IndicBERT</em>) [IndicNLPSuite: Monolingual Corpora, Evaluation Benchmarks and Pre-trained Multilingual Language Models for Indian Languages](https://aclanthology.org/2020.findings-emnlp.445/) (Kakwani et al., Findings 2020)

* (<em>XLM-R</em>) [Unsupervised Cross-lingual Representation Learning at Scale](https://aclanthology.org/2020.acl-main.747/) (Conneau et al., ACL 2020)


In [15]:
# all models we will use
all_models = {  "AxomiyaBERTa" : "Abhijnan/AxomiyaBERTa", #
                "IndicBERT" : "ai4bharat/indic-bert",
                "XLM-R" : "FacebookAI/xlm-roberta-base"}

# average surprisal for sections
average_surprisal = {"definiteness": {}, "classifiers": {}}
# average surprisal for incorrect sentences in surprisals
average_surprisal_incor = {"definiteness": {}, "classifiers": {}}

# average relative change for sections
surpavrat = {"definiteness": {}, "classifiers": {}}

for subexp, language_documents in (("definiteness",assamese_contextual_sentences), 
                                   ("classifiers",assamese_contextual_cls_sentences)):
    
    print(f"Now doing: {subexp}")
    # average relative change for this model
    surprisal_avg_rats = surpavrat[subexp]
    
    # average surprisal of correct answers generally for this model
    avg_surprisal = average_surprisal[subexp]
    # average surprisal of incorrect answers generally for this model
    avg_surprisal_incor = average_surprisal_incor[subexp]

    # iterate thorugh models
    for (model_name, model_path) in all_models.items():
        print(f'Loading model "{model_name}".')

        # use minicons scorer for surprisal
        scorer_model = scorer.MaskedLMScorer(model_path, "cpu")

        # number of contexts with the sentence that has min surprisal being the most grammatically correct option 
        surprisal_ratios = 0
        average_surprisal_vals = 0
        average_surprisal_vals_incor = 0

        # get each set of sentences with their context
        for (context, sentences) in language_documents.items():

            context_blank = None 
            if (type(context) != tuple): # check if our context is supposed to be blank
                context = ("(no context)", f"({context})")
                context_blank = context
            
            #print(f'  Context sentence: \"{context[0]}\" (translation:\"{context[1]}\")')

            # list of (sentence, translation, 1 or 0)
            ctxt_sents_transes_nums = [(context, sent_trans_num) for sent_trans_num in sentences]

            # conjoin sentence of context with completion 
            # (the models don't seem to support sequencing surprisal 
            # so we will just concatenate the sentences together directly, unless context is '\nil')
            documents_for_model = [(("" if context_blank else (ctxt[0] + " ")) + s_t_r[0]) for (ctxt, s_t_r) in ctxt_sents_transes_nums]

            # get surprisal values
            output = scorer_model.sequence_score(documents_for_model, reduction = lambda x: -x.sum(0).item())

            # put the sentences and their surprisals in a dictionary
            # dict of {(sent, trans) : output}
            s_t_output_dict = (dict(zip([(s_t_r[0],s_t_r[1]) for (c, s_t_r) in ctxt_sents_transes_nums], output)))
            #print("\t> Surprisal values for each sentence (alongside translations)=\n\t\t" + '\n\t\t'.join([f"'{sent}' (trans:'{trans}'): {surp}" for ((sent, trans), surp) in surprisal_dict.items()]))
            
            # sentence which should be correct
            correct_s_t = max([s for (c, s) in ctxt_sents_transes_nums], key=operator.itemgetter(2))[0:2]

            # sentence which should be incorrect
            incorrect_s_t = min([s for (c, s) in ctxt_sents_transes_nums], key=operator.itemgetter(2))[0:2]

            # get the sentence that got the minimum surprisal in this context
            #min_surprisal_st = min(s_t_output_dict.items(), key=operator.itemgetter(1))[0]
            #print(f"\t> Sentence with minimum surprisal for this context={min_surprisal_st} ({s_t_output_dict[min_surprisal_st]})")
            
            # get which completion we expected to get minimum surprisal (i.e. the "most correct" one)
            #print(f"\t> Sentence expected to have min surprisal={correct_s_t} ({s_t_output_dict[correct_s_t]})")
            
            # add it to the count
            cor_surp = s_t_output_dict[correct_s_t]
            incor_surp = s_t_output_dict[incorrect_s_t]
            average_surprisal_vals += cor_surp
            average_surprisal_vals_incor += incor_surp
            surprisal_ratios += (incor_surp - cor_surp) / ((cor_surp + incor_surp)/2)
            #print(f" {cor_surp} for correct, {incor_surp} for incorrect, rel_change = {(incor_surp - cor_surp) / ((cor_surp + incor_surp)/2)}")
        
        # store proportion of "correct values"
        surprisal_avg_rats[model_name] = surprisal_ratios / len(language_documents)
        avg_surprisal[model_name] = average_surprisal_vals / len(language_documents)
        avg_surprisal_incor[model_name] = average_surprisal_vals_incor / len(language_documents)

Now doing: definiteness
Loading model "AxomiyaBERTa".
Loading model "IndicBERT".
Loading model "XLM-R".
Now doing: classifiers
Loading model "AxomiyaBERTa".
Loading model "IndicBERT".
Loading model "XLM-R".


In [9]:
# print out the proportion of correct minimum surprisals
print(surpavrat)

{'definiteness': {'AxomiyaBERTa': 0.00324125829327274, 'IndicBERT': 0.006940457866722931, 'XLM-R': -0.02022868300417183}, 'classifiers': {'AxomiyaBERTa': 0.030260501309888686, 'IndicBERT': 0.002088649027089393, 'XLM-R': 0.06193068581282691}}


In [17]:
print(average_surprisal)
print(average_surprisal_incor)

{'definiteness': {'AxomiyaBERTa': 206.16879163469588, 'IndicBERT': 97.46234893798828, 'XLM-R': 44.38074248177664}, 'classifiers': {'AxomiyaBERTa': 125.08082266414867, 'IndicBERT': 87.94368474623737, 'XLM-R': 38.93951404795927}}
{'definiteness': {'AxomiyaBERTa': 205.61591175624304, 'IndicBERT': 97.74071012224469, 'XLM-R': 43.72855595179966}, 'classifiers': {'AxomiyaBERTa': 127.80340800565831, 'IndicBERT': 88.00826532700482, 'XLM-R': 41.2108814015108}}
